## Part 1: Pulse Rate Algorithm

### Contents
Fill out this notebook as part of your final project submission.

**You will have to complete both the Code and Project Write-up sections.**
- The [Code](#Code) is where you will write a **pulse rate algorithm** and already includes the starter code.
   - Imports - These are the imports needed for Part 1 of the final project. 
     - [glob](https://docs.python.org/3/library/glob.html)
     - [numpy](https://numpy.org/)
     - [scipy](https://www.scipy.org/)
- The [Project Write-up](#Project-Write-up) to describe why you wrote the algorithm for the specific case.


### Dataset
You will be using the **Troika**[1] dataset to build your algorithm. Find the dataset under `datasets/troika/training_data`. The `README` in that folder will tell you how to interpret the data. The starter code contains a function to help load these files.

1. Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Link

-----

### Code

In [1]:
import glob

import numpy as np
import scipy as sp
import scipy.io
from scipy import signal

# set the sampling rate to 125Hz
fs = 125

# windowing parameters (win_length_s = 8s and win_shift_s = 2s)
win_length_s  = 8  
win_shift_s = 2 

win_length = win_length_s * fs
win_shift = win_shift_s * fs

# bpm boundary
lower_bpm, high_bpm = 40, 240

# frequency bins boundary
low, high = lower_bpm/60, high_bpm/60

def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]

def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)

def ground_truth(ref_fl):
    """ loads the ground truth values    
    args:
        ref_fl: Name of the .mat file that contains reference data    
    Returns:
        A numpy array of the ground truth values for the input ref_fl
    """
    target = sp.io.loadmat(ref_fl)['BPM0']    
    return target
    
def bandpass_filter(signal, fs, low, high):
    """ 
    filters the input signal between cut-off values.   
    Args:
        signal: A numpy array representing the signal to filter
        fs: sampling rate
        low = required lower bound of frequency bin
        high = required upper bound of frequency bin        
    Returns:
        A numpy array of the filtered signal
    """ 
    b, a = sp.signal.butter(3, (low, high), btype='bandpass', fs=fs)
    return sp.signal.filtfilt(b, a, signal)

def feature_generator(signal, fs):
    """
    uses the input signals to generate features within the specified windows
    Args:
        signal: bandpass filtered ppg signal             
        fs: sampling rate        
    Returns:
        tuple of features derived from the input signals
        
        The derived features are 
         1. peaks
         2. frequencies at the peaks
         3. frequency of the signal, and 
         4. the fourier transform of the signal
         5. mean of the signal
         6. energy of the signal
    """ 

    # fft of ppg signal
    fft_len = 2*len(signal)
    freq = np.fft.rfftfreq(fft_len, 1/fs)
    fft = np.abs(np.fft.rfft(signal, fft_len))
    
    # Filter fft between low_bpm and high_pbm
    fft[freq <= 70/60] = 0.0 # changed lower_bpm to 70 instead of 40 to bring the error down
    fft[freq >= 190/60] = 0.0 # changed lower_bpm to 190 instead of 240 for the same reason
    
    # ppg peaks and peak frequencies
    peaks = sp.signal.find_peaks(fft, height=2000, distance = 10)[0]
    peaks_freq = freq[peaks]
    
    # signal mean  and energy
    signal_filtered = bandpass_filter(signal, fs, low, high)
    mean = np.mean(signal_filtered)
    energy = np.sum(np.square(signal_filtered - mean))

    return freq, fft, peaks, peaks_freq, mean, energy


def RunPulseRateAlgorithm(data_fl, ref_fl):
    """
    Uses the data and reference files, computes pulse rate estimates and returns errors and confidence.    
    Args:
        data_fl: Data file containing input ppg and accelerometer signals
        ref_fl: refernece file containing ground truth values    
    Returns:
        Numpy arrays of errors and confidence computed at 8s window
    """
    
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    
    # load the ground truth    
    target = ground_truth(ref_fl)
    
    # filter signals
    ppg = bandpass_filter(ppg, fs, low, high)
    accx = bandpass_filter(accx, fs, low, high)
    accy = bandpass_filter(accy, fs, low, high)
    accz = bandpass_filter(accz, fs, low, high)

    acc = np.sqrt(accx**2 + accy**2 + accz**2)
    
    # compute pulse rates
    tol = 0.25
    est, confidence = [],[]
   
    for i in range(0, len(ppg) - win_length, win_shift):
        
        ppg_freq, ppg_fft, ppg_peaks, ppg_peaks_freq, mean_ppg, energy_ppg = feature_generator(ppg[i:i+win_length], fs)
        acc_freq, acc_fft, acc_peaks, acc_peaks_freq, mean_acc, energy_acc = feature_generator(acc[i:i+win_length], fs)

        max_ppg = ppg_freq[np.argmax(energy_ppg)]
        max_acc = acc_freq[np.argmax(energy_acc)]
        
        j = 1        
        while np.abs(max_ppg-max_acc) <= tol and j <=2:
            j+=1
            max_ppg = ppg_freq[np.argsort(ppg_fft, axis=0)[-j]]
            max_acc = acc_freq[np.argsort(acc_fft, axis=0)[-j]]

        est_freq = max_ppg
        est.append(est_freq*60)
        
        # compute confidence
        win_freq = 30/60
        conf = np.sum(ppg_fft[(ppg_freq >= est_freq - win_freq) & (ppg_freq <= est_freq + win_freq)])/ np.sum(ppg_fft)                
        confidence.append(conf)    
        
    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    errors = np.abs(np.diag(est-target))
    return errors, np.array(confidence)

In [2]:
Evaluate()

ValueError: need at least one array to concatenate

-----
### Project Write-up

Answer the following prompts to demonstrate understanding of the algorithm you wrote for this specific context.

> - **Code Description** - Include details so someone unfamiliar with your project will know how to run your code and use your algorithm. 
> - **Data Description** - Describe the dataset that was used to train and test the algorithm. Include its short-comings and what data would be required to build a more complete dataset.
> - **Algorithhm Description** will include the following:
>   - how the algorithm works
>   - the specific aspects of the physiology that it takes advantage of
>   - a describtion of the algorithm outputs
>   - caveats on algorithm outputs 
>   - common failure modes
> - **Algorithm Performance** - Detail how performance was computed (eg. using cross-validation or train-test split) and what metrics were optimized for. Include error metrics that would be relevant to users of your algorithm. Caveat your performance numbers by acknowledging how generalizable they may or may not be on different datasets.

Your write-up goes here...

**Code description**

The code is composed of different functions which work together to estimate the errors and confidence associated with heart rate estimations from photoplethysmogram (ppg) and accelerometer signals and compared to some reference heart rates. 

To use the code, simply run the **Evaluate()** function which calls the functions to load the TROIKA datasets, extracts the signals, applies bandpass filters to them, generates features and then runs the **RunPulseRateAlgorithm()** to estimate the heart rates, and then computes the errors between the estimated heart rates and the reference values and also calculated the associated confidences. Lastly, it aggregate the errors by calling the **AggregateErrorMetric()**.


**Data description**

The datasets used were sourced from Zhang et. al (2015) and are composed on ppg and three accelerometer signals in x-, y- and z- directions sampled 125Hz. The datasets were acquired from subjects aged 18 - 35 years. The reference heart rates used as  the ground truth were ECG acquired in 8s window simultaneously as the other signals.

> Activities: rest for 30s followed by run for 4 minutes on a treadmill with speed varied every 1 minute and lastly, rest for 30s

> Features of Sensor Used: All the signals were sampled at 125Hz. The ppg signals were acquired from the wrist using two pulse oximeters with green LEDS of wavelength 515nm separated by a 2cm distance and accelerometer signals were acquired from the wrist in three axis. The ECG signals were acquired in 8s window simultaneously as the other signals

> Quantity of data in the dataset: dataset contains 12 data files and 12 reference files (from 11 subjectes). Each of the data files contains 6 rows representing ECG, 2 channels of PPG and 3 recording of 3-dimensional accelerometer signals. 

> Shortcomings of the dataset: dataset was based on limited fixed activities (as above), the experiment was not performed in a free-style context that could mimic the wearer's daily routine.  


**Algorithm description**

As discussed under the code description, the core of the algorithm is the **RunPulseRateAlgorithm()** function which estimates the heart rates, compares them with the gorund truth, computes the associated errors and confidence for a given data_file and reference_file. The errors are aggregated for all the datafiles and reference files by the **AggregateErrorMetric()** function which returns the mean error of the best pulse rate estimates (@ 90% percentile). Both of these functions are called by the **Evaluate()**. 

The **LoadTroikaDataFile(data_fl)** and the **ground_truth(ref_fl)** functions load the input signals and reference heart rates respectively for each pair of locations returned by the **LoadTroikaDataset()** function. The **bandpass_filter()** filters the input signals within the specified limits of 40Hz and 240Hz. The **feature_generator(signal, fs)** function generates features such as Fourier transforms of the signals, their peak values, the frequencies of the peaks, etc. from the bandpass filtered signals.

> Description of the Physiology: The PPG sensor is equipped with LEDs that emits (green) lights which are absorbed by red-blood cells as blood flows in and out of the capillaries in the wrist. The amount of reflected light detected by the photodetector in the ppg sensor changes as the volume of bloods into the capillaries in the wrist changes during each phase of a heart-beat resulting in an oscillating waveform, the period of which is the pulse rate. Arm swings associated with the wearer's movements can also be seen as periodic signals on the PPG sensor. Such signals can be tracked using accelerometer which only senses arm motion.

> An intuitive explanation of why the confidence algorithm produces a higher number for better estimates: 
The confidence algorithm produces a higher number for better estimates because we are able to pick strong frequencies of the ppg signal that not present in the accelerometer at the better/best estimates. Since confidence interval was calculated as the sum of the magnitude of the estimated 'strong' frequency spectrum divided by the sum of the magnitude of the entire frequency spectrum of the ppg signals, the more strong frequencies we picked, the higher the confidence level and the better the pulse rate estimates.   


> Insightful caveats and failure-modes of the algorithm: The algorithm is based on estimation of strong frequency spectral in the ppg signals relative to the entire frequency spectral of the ppg signals. The wearer's motion can produce strong signals which can be confused for the pulse rate resulting in algorithm failure.  


**Algorithm performance**

The mean absolute error (MAE) at 90% availability using the confidence estimates for best estimates. The MAE was calculated by comparing  heart rates estimated in 8s window with the given reference values (ground) in the same window length. The value of the error was 13.63 which was below the 15bpm required margin. 

The performance metric reported here was based on a heuristic model built on a limited dataset. Thus, it may not be generalisable to handle other datasets.



**References**
> (1). Zhilin Zhang, Zhouyue Pi, Benyuan Liu, ‘‘TROIKA: A General Framework for Heart Rate Monitoring Using Wrist-Type
Photoplethysmographic Signals During Intensive Physical Exercise,’’IEEE Trans. on Biomedical Engineering, vol. 62, no. 2, pp. 522-531, February 2015. Available on https://ieeexplore.ieee.org/document/6905737

-----
### Next Steps
You will now go to **Test Your Algorithm** to apply a unit test to confirm that your algorithm met the success criteria. 